# Работа с датами в DataFrame

In [56]:
import pandas as pd
from IPython.display import display
melb_data = pd.read_csv('data/melb_data_ps.csv', sep=',')
melb_df = melb_data.copy()
melb_df = melb_df.drop(['index', 'Coordinates'], axis=1)
total_rooms = melb_df['Rooms'] + melb_df['Bedroom'] + melb_df['Bathroom']
melb_df['MeanRoomsSquare'] = melb_df['BuildingArea'] / total_rooms
diff_area = melb_df['BuildingArea'] - melb_df['Landsize']
sum_area = melb_df['BuildingArea'] + melb_df['Landsize']
melb_df['AreaRatio'] = diff_area/sum_area

##### ПРИЗНАКИ ДАТЫ И ВРЕМЕНИ

Единый способ обозначения даты и времени в Pandas является формат **datetime**, который записывается как 

YYYY-MM-DD HH: MM: SS,

 то есть составляющие времени указываются в следующем порядке: год, месяц, день, час, минута, секунда

In [57]:
display(melb_df['Date'])

0         3/12/2016
1         4/02/2016
2         4/03/2017
3         4/03/2017
4         4/06/2016
            ...    
13575    26/08/2017
13576    26/08/2017
13577    26/08/2017
13578    26/08/2017
13579    26/08/2017
Name: Date, Length: 13580, dtype: object

Для того чтобы преобразовывать столбцы с датами, записанными в распространённых форматах, в формат datetime, можно воспользоваться функцией

**pandas.to_datetime()**

В нашем случае в функции нужно указать параметр dayfirst=True, который будет обозначать, что в первоначальном признаке первым идет день. Преобразуем столбец Date в формат datetime, передав его в эту функцию

In [58]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
display(melb_df['Date'])

0       2016-12-03
1       2016-02-04
2       2017-03-04
3       2017-03-04
4       2016-06-04
           ...    
13575   2017-08-26
13576   2017-08-26
13577   2017-08-26
13578   2017-08-26
13579   2017-08-26
Name: Date, Length: 13580, dtype: datetime64[ns]

Изменился тип данных для столбца Date, теперь его тип — datetime64

##### ВЫДЕЛЕНИЕ АТРИБУТОВ DATETIME

Тип данных datetime позволяет с помощью специального **аксессора dt** выделять составляющие времени из каждого элемента столбца, такие как:

* date — дата;
* year, month, day — год, месяц, день;
* time — время;
* hour, minute, second — час, минута, секунда;
* dayofweek — номер дня недели, от 0 до 6, где 0 — понедельник, 6 — воскресенье;
* day_name — название дня недели;
* dayofyear — порядковый день года;
* quarter — квартал (интервал в три месяца).

Например, обратившись по атрибуту dt.year в столбце Date, мы можем «достать» год продажи и понять, за какой интервал времени (в годах) представлены наши данные, а также на какой год приходится наибольшее число продаж

In [59]:
years_sold = melb_df['Date'].dt.year
print(years_sold)
print('Min year sold:', years_sold.min())
print('Max year sold:', years_sold.max())
print('Mode year sold:', years_sold.mode()[0]) # метод mode() возвращает объект Series, поэтому берем первое значение [0]

0        2016
1        2016
2        2017
3        2017
4        2016
         ... 
13575    2017
13576    2017
13577    2017
13578    2017
13579    2017
Name: Date, Length: 13580, dtype: int64
Min year sold: 2016
Max year sold: 2017
Mode year sold: 2017


Теперь попробуем понять, на какие месяцы приходится пик продаж объектов недвижимости. Для этого выделим атрибут dt.month и на этот раз занесём результат в столбец MonthSale

In [60]:
melb_df['MonthSale'] = melb_df['Date'].dt.month

Затем найдём относительную частоту продаж для каждого месяца от общего количества продаж — для этого используем метод value_counts() с параметром normalize (вывод в долях)

In [61]:
melb_df['MonthSale'].value_counts(normalize=True)

5     0.149411
7     0.145950
9     0.135862
6     0.134757
8     0.114138
11    0.082032
4     0.069882
3     0.049926
12    0.044698
10    0.040574
2     0.032622
1     0.000147
Name: MonthSale, dtype: float64

Из результатов становится ясно, что наибольшее количество продаж недвижимости приходится на май, июль и сентябрь (пятый, седьмой и девятый месяцы соответственно). Месяцами застоя при этом являются месяцы — октябрь, февраль и январь (десятый, второй и первый месяцы соответственно)

##### РАБОТА С ИНТЕРВАЛАМИ

Чтобы узнать количество времени (дни, например), которое прошло от выбранного времени (переменная) до момента продажи (значение в таблице), можно просто найти разность между ними

In [62]:
delta_days = melb_df['Date'] - pd.to_datetime('2016-01-01') 
display(delta_days)

0       337 days
1        34 days
2       428 days
3       428 days
4       155 days
          ...   
13575   603 days
13576   603 days
13577   603 days
13578   603 days
13579   603 days
Name: Date, Length: 13580, dtype: timedelta64[ns]

В результате мы получаем Series, элементами которой является количество дней, которое прошло с 1 января 2016 года, данные такого формата относятся к типу timedelta. Переведём в формат чисел:

In [63]:
display(delta_days.dt.days)

0        337
1         34
2        428
3        428
4        155
        ... 
13575    603
13576    603
13577    603
13578    603
13579    603
Name: Date, Length: 13580, dtype: int64

In [64]:
# Возраст объекта на момент продажи (разность года продажи и постройки) сразу вписываем в таблицу
melb_df['AgeBuilding'] = melb_df['Date'].dt.year - melb_df['YearBuilt']
# Выводим в виде столбца AgeBuilding
display(melb_df['AgeBuilding'])

0         46
1        116
2        117
3         47
4          2
        ... 
13575     36
13576     22
13577     20
13578     97
13579     97
Name: AgeBuilding, Length: 13580, dtype: int64

Так как года кодируются целым числом, в результате мы тоже получаем целочисленный столбец — тип int64 (а не timedelta)

Поскольку зная год постройки здания, мы автоматически знаем его возраст, следует оставить только возраст здания, так как он является более наглядным, а год постройки из таблицы удалить

In [65]:
melb_df = melb_df.drop('YearBuilt', axis=1)

Задачи:

In [66]:
# Создайте в таблице melb_df признак WeekdaySale (день недели).
# Найдите, сколько объектов недвижимости было продано в выходные (суббота и воскресенье),
# результат занесите в переменную weekend_count
melb_df['WeekdaySale'] = melb_df['Date'].dt.dayofweek
weekend_count = melb_df['WeekdaySale'][melb_df['WeekdaySale'] > 4].shape[0]
print(weekend_count)

12822


In [67]:
# Представлены данные (в формате csv) об отчётах очевидцев НЛО в США за период с 1930 по 2020 год
ufo_data = pd.read_csv('data/ufo.csv', sep=',')
ufo_data.head()

,City,Colors Reported,Shape Reported,State,Time
0,Ithaca,NaN,TRIANGLE,NY,6/1/1930 22:00
1,Willingboro,NaN,OTHER,NJ,6/30/1930 20:00
2,Holyoke,NaN,OVAL,CO,2/15/1931 14:00
3,Abilene,NaN,DISK,KS,6/1/1931 13:00
4,New York Worlds Fair,NaN,LIGHT,NY,4/18/1933 19:00


In [68]:
# В каком году отмечается наибольшее количество случаев наблюдения НЛО в США
ufo_data_year = pd.to_datetime(ufo_data['Time']).dt.year
display(ufo_data_year.mode())

0    1999
Name: Time, dtype: int64

In [69]:
# Найдите средний интервал времени (в днях) между двумя
# последовательными случаями наблюдения НЛО в штате Невада (NV)
ufo_data['ufo_data_date'] = pd.to_datetime(ufo_data['Time'])
ufo_data_date_NV = ufo_data[ufo_data['State'] == 'NV']['ufo_data_date'].dt.date
display(ufo_data_date_NV.diff().mean())

Timedelta('68 days 22:18:13.992932862')

ЗАПИСЬ В CSV

In [70]:
melb_df.to_csv('data/melb_data_ps.csv', index=False, sep=',')